In [1]:
import findspark
findspark.init("/home/zshnhaque/Downloads/spark2")
import pyspark
import numpy as np
from pyspark.sql.functions import lit
from pyspark.sql.functions import col 
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [2]:
sc = pyspark.SparkContext(appName="BDAP_spark")

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
import pandas as pd

In [5]:
trainDF = spark.read.csv("/home/zshnhaque/Downloads/train.csv", header="true")

In [6]:
req = pd.read_csv("/home/zshnhaque/Downloads/train.csv")

In [7]:
col_names = req.columns
catg_var = []
int_var = []
float_var = []
for i in range(0,len(col_names)):
    if req[col_names[i]].dtype == 'int64':
        int_var.append(col_names[i])
    elif req[col_names[i]].dtype == 'float64':
        float_var.append(col_names[i])
    else:
        catg_var.append(col_names[i])

In [8]:
def to_anytype(df, colnames, typename):
    for colname in colnames:
        df = df.withColumn("tmp", df[colname].cast(typename)) \
        .drop(colname) \
        .withColumnRenamed("tmp", colname)
    return(df)


In [9]:
trainDF = to_anytype(trainDF, int_var, "integer")
trainDF = to_anytype(trainDF, catg_var, "string")
trainDF = to_anytype(trainDF, float_var, "float")

In [10]:
testDF = spark.read.csv("/home/zshnhaque/Downloads/test.csv", header="true")

In [11]:
catg_var1 = []
int_var1 = []
float_var1 = []
for i in range(0,len(col_names)-1):
    if req[col_names[i]].dtype == 'int64':
        int_var1.append(col_names[i])
    elif req[col_names[i]].dtype == 'float64':
        float_var1.append(col_names[i])
    else:
        catg_var1.append(col_names[i])

In [12]:
testDF = to_anytype(testDF, int_var1, "integer")
testDF = to_anytype(testDF, catg_var1, "string")
testDF = to_anytype(testDF, float_var1, "float")

In [26]:
def indexer(df, col):
    si = StringIndexer(inputCol = col, outputCol = col+'_indexed').fit(df)
    return si
 
indexers = [indexer(trainDF, col) for col in catg_var]
pipeline = Pipeline(stages = indexers)
train_indexed = pipeline.fit(trainDF).transform(trainDF)

In [14]:
indexers = [indexer(testDF, col) for col in catg_var1]
pipeline = Pipeline(stages = indexers)
test_indexed = pipeline.fit(testDF).transform(testDF)

In [15]:
del req

In [16]:
del trainDF

In [17]:
del testDF

In [16]:
from pyspark.sql import Row
from pyspark.ml.linalg import DenseVector

In [17]:
catg_var_indexed = [i + '_indexed' for i in catg_var]

In [18]:
all_feature_col = catg_var_indexed+float_var1

In [19]:
id_col_train = int_var

In [20]:
target_col = ['loss']

In [21]:
train_indexed = train_indexed[id_col_train+target_col+all_feature_col]

In [22]:
row = Row('id','loss','features') 

In [23]:
train = train_indexed.rdd.map(lambda r: (row(r[0], r[1],DenseVector(r[2:])))).toDF()

In [24]:
from pyspark.ml.regression import GBTRegressor

In [33]:
gbt_model = GBTRegressor(featuresCol="features",labelCol="loss", maxIter=50).fit(train)

In [34]:
test_indexed = test_indexed[id_col_train+all_feature_col]

In [35]:
row1= Row('id','features') 

In [36]:
test = test_indexed.rdd.map(lambda r: (row1(r[0],DenseVector(r[1:])))).toDF()

In [37]:
predictions = gbt_model.transform(test)

In [43]:
pred = predictions.selectExpr('id','prediction as loss')

In [44]:
pred.toPandas().to_csv("/home/zshnhaque/Downloads/submission_file.csv")